## LSTM class Developed
-----------------------------
[feat] 7/16
- ! to prevent overfitting use K-fold, validation split ! <- NEED TO DEVELOP
- basical LSTM model => lstm dense need to doubled
- not tested
- without AOC, interest rate
- bidirectional LSTM Model

imorts & get datas
------------

In [ ]:
#!pip install -U finance-datareader

#%pip install FinanceDataReader

In [19]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as pdr
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense ,LSTM, Dropout,Bidirectional
from sklearn.model_selection import train_test_split, KFold
from tensorflow import expand_dims
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from os.path import join
from tensorflow import constant
#import FinanceDataReader as fdr

In [2]:
data=pd.read_csv('/content/drive/MyDrive/2023_1st_vac/KRX_DATA/train.csv')

In [3]:
# prescaler
def getAOC(data):
  AOC = []
  close = data['종가']
  close_pre = close.shift(1)

  AOC.append(0)

  # aoc = (금일 종가 - 전일 종가) / 전일 종가
  for i in range(1, len(close)):
    aoc_value = ((close[i] - close_pre[i]) / close_pre[i]) * 100
    AOC.append(aoc_value)

  return AOC

#sort data by "종목명"
subject_Data=data.sort_values(by="종목명",inplace=False)
subject_Data.head()
subject_Data.columns

pv_s_data=pd.pivot_table(subject_Data,values=['거래량', '시가', '고가', '저가', '종가'],index=['종목코드','일자'])

divided = pv_s_data.index.get_level_values('종목코드').unique()

sliced_dataframes = []

for i in divided:
    sliced_data = pv_s_data.loc[i]
    sliced_dataframe = sliced_data.reset_index()
    sliced_dataframes.append(sliced_dataframe)

In [ ]:
data=sliced_dataframes
data

In [7]:
data[0].shape

(494, 6)

In [62]:
class LSTM_Object():

  def __init__(self,data):
    #get sliced data
    self.data=data

  def modeling(self):

    self.prescale()
    self.model=Sequential()
    #layers
    self.model.add(LSTM(1024,return_sequences=True,input_shape=(self.x_train.shape[1], self.x_train.shape[2])))
    self.model.add(LSTM(1024,return_sequences=True,input_shape=(self.x_train.shape[1], self.x_train.shape[2])))
    self.model.add(Dropout(0.25))
    self.model.add(LSTM(1024,return_sequences=True,input_shape=(self.x_train.shape[1], self.x_train.shape[2])))
    self.model.add(LSTM(1024,return_sequences=True,input_shape=(self.x_train.shape[1], self.x_train.shape[2])))
    self.model.add(Dropout(0.25))
    self.model.add(LSTM(1024,return_sequences=True,input_shape=(self.x_train.shape[1], self.x_train.shape[2])))
    #output layer
    self.model.add(Dense(1, activation='sigmoid'))

    self.filename = join('tmp', 'ckeckpointer.ckpt')
    checkpoint = ModelCheckpoint(self.filename, save_weights_only=True, save_best_only=True,monitor='val_loss',verbose=1)

    earlystopping = EarlyStopping(monitor='val_loss', patience=100)


    self.model.compile(loss = 'binary_crossentropy', optimizer=Adam(learning_rate=0.001))
    history=self.model.fit(self.x_train,self.y_train,epochs=1000,batch_size=128,validation_data=(self.x_train,self.y_train),shuffle=False,callbacks=[checkpoint, earlystopping])


  def prescale(self):
    x_scaler=MinMaxScaler()
    y_scaler=MinMaxScaler()
    self.data[['거래량', '고가', '시가', '저가']]=x_scaler.fit_transform(self.data[['거래량', '고가', '시가', '저가']])
    self.data['종가']=y_scaler.fit_transform(self.data['종가'].values.reshape(-1,1))
    x_data=self.data[['거래량', '고가', '시가', '저가']]
    y_data=self.data['종가']
    x_train,x_test,y_train,y_test=train_test_split(x_data,y_data,test_size=0.2)

    ### reshape input datas()

    input_dim = constant(2)
    self.x_train=expand_dims(x_train,input_dim)
    self.x_test=expand_dims(x_test,input_dim)
    self.y_train=y_train
    self.y_test=y_test

  def ret_learning(self):
    self.prescale()
    self.modeling()
    self.model.load_weights(self.filename)
    pred = self.model.predict(self.x_test)
    plt.figure(figsize=(12, 9))
    plt.plot(np.asarray(self.y_test)[20:], label='actual')
    plt.plot(pred, label='prediction')
    plt.legend()
    plt.show()

In [63]:
test_1=data[0]

In [64]:
L=LSTM_Object(test_1)
L.prescale()

In [ ]:
L.ret_learning()

Epoch 1/1000
3/4 [=====================>........] - ETA: 0s - loss: 0.6876
Epoch 1: val_loss improved from inf to 0.65558, saving model to tmp/ckeckpointer.ckpt
4/4 [==============================] - 15s 2s/step - loss: 0.6870 - val_loss: 0.6556
Epoch 2/1000
3/4 [=====================>........] - ETA: 0s - loss: 0.6494
Epoch 2: val_loss improved from 0.65558 to 0.63669, saving model to tmp/ckeckpointer.ckpt
4/4 [==============================] - 2s 688ms/step - loss: 0.6481 - val_loss: 0.6367
Epoch 3/1000
3/4 [=====================>........] - ETA: 0s - loss: 0.6350
Epoch 3: val_loss improved from 0.63669 to 0.62107, saving model to tmp/ckeckpointer.ckpt
4/4 [==============================] - 3s 1s/step - loss: 0.6343 - val_loss: 0.6211
Epoch 4/1000
3/4 [=====================>........] - ETA: 0s - loss: 0.6064
Epoch 4: val_loss improved from 0.62107 to 0.58219, saving model to tmp/ckeckpointer.ckpt
4/4 [==============================] - 5s 2s/step - loss: 0.6047 - val_loss: 0.5822
Epoc